In [6]:
from playwright.async_api import async_playwright, Playwright
from rich import print
import asyncio

In [4]:
from playwright.async_api import async_playwright
from collections import defaultdict
from urllib.parse import urljoin
import nest_asyncio
import json
import re
nest_asyncio.apply()
from collections import defaultdict
import re
from urllib.parse import urljoin

async def run(playwright):
    start_url = "https://aniagotuje.pl"
    browser = await playwright.chromium.launch(headless=True)
    context = await browser.new_context()
    page = await context.new_page()
    await page.goto(start_url)

    # Dodatkowa, JEDNA strona do scrapowania przepisów
    scraper_page = await context.new_page()

    recipecount = 0
    recipemap = defaultdict(lambda: {"ingredient": [], "recipe_info": []})
    recipemap_end = defaultdict(lambda: {"ingredient": [], "recipe_info": []})

    while True:
        if recipecount < 100:
            links = await page.locator('a[href^="/przepis/"]').all()
            if len(links) < 20:
                break
            print(f"Znaleziono {len(links)} linków")

            linkcount = 0

            for link in links:
                if linkcount >= 30:
                    break

                href = await link.get_attribute('href')
                if not href or "#" in href or href in recipemap or href in recipemap_end:
                    continue

                full_url = urljoin(start_url, href)

                try:
                    await scraper_page.goto(full_url, wait_until="domcontentloaded")
                    print(full_url)

                    ingredients = await scraper_page.locator('span[class^="ingredient"]').all()
                    qty = await scraper_page.locator('span[class^="qty"]').all()
                    recipe_info = await scraper_page.locator('p[class^="recipe-info"]').all()

                    if len(qty) > 0:
                        for i in range(len(qty)):
                            text_ingredients = await ingredients[i].text_content()
                            text_qty = await qty[i].text_content()
                            recipemap_end[href]["ingredient"].append({
                                "name": text_ingredients,
                                "amount": text_qty
                            })
                        if len(ingredients) > len(qty):
                            for ing in ingredients[len(qty):]:
                                text = await ing.text_content()
                                recipemap_end[href]["ingredient"].append({
                                    "name": text,
                                    "amount": ""
                                })
                        for inf in recipe_info:
                            inftext = await inf.text_content()
                            inftext = inftext.replace("\t", " ")
                            inftext = re.sub(r'\s{2,}', ",", inftext)
                            recipemap_end[href]["recipe_info"].append(inftext)
                    else:
                        for ing in ingredients:
                            text = await ing.text_content()
                            recipemap[href]["ingredient"].append(text)
                        for inf in recipe_info:
                            inftext = await inf.text_content()
                            inftext = inftext.replace("\t", " ")
                            inftext = re.sub(r'\s{2,}', ",", inftext)
                            recipemap[href]["recipe_info"].append(inftext)

                except Exception as e:
                    print(f"Error scraping {href}: {e}")
                    continue

                linkcount += 1

        else:
            break

        recipecount += 1

        # Następna strona
        next_link = page.locator('a.page-link', has_text='»')
        if await next_link.count() > 0:
            href = await next_link.get_attribute('href')
            if href:
                full_url = urljoin(start_url, href)
                await page.goto(full_url, wait_until='load')
            else:
                break
        else:
            break

    await browser.close()
    return recipemap, recipemap_end



async with async_playwright() as playwright:
    mappa,mappa_end = await run(playwright)

    with open("przepisy.json", "w", encoding="utf-8") as f:
        json.dump(mappa, f, ensure_ascii=False, indent=2)
    with open("przepisy_end.json", "w", encoding="utf-8") as f:
        json.dump(mappa_end, f, ensure_ascii=False, indent=2)

Znaleziono 70 linków
https://aniagotuje.pl/przepis/jak-zrobic-ciasto-na-nalesniki
https://aniagotuje.pl/przepis/placki-z-cukinii
https://aniagotuje.pl/przepis/ogorki-konserwowe
https://aniagotuje.pl/przepis/ogorki-kiszone
https://aniagotuje.pl/przepis/sos-kurkowy
https://aniagotuje.pl/przepis/gofry
https://aniagotuje.pl/przepis/leczo
https://aniagotuje.pl/przepis/salatka-z-ogorkow
https://aniagotuje.pl/przepis/szarlotka
https://aniagotuje.pl/przepis/ogorki-z-chili
https://aniagotuje.pl/przepis/jagodzianki
https://aniagotuje.pl/przepis/ciasto-kruche-ze-sliwkami
https://aniagotuje.pl/przepis/szybkie-ciasto-ze-sliwkami-na-duza-blache
https://aniagotuje.pl/przepis/tosty-mleczne
https://aniagotuje.pl/przepis/sernik-czekoladowy-z-malinami
https://aniagotuje.pl/przepis/nalesnik-pieczony-ze-sliwkami
https://aniagotuje.pl/przepis/pasta-z-bobu-i-jajek
https://aniagotuje.pl/przepis/ciasto-ze-sliwkami-i-kruszonka
https://aniagotuje.pl/przepis/fruzelina-malinowa
https://aniagotuje.pl/przepis/lody-z

In [25]:
print(mappa_end)

defaultdict(<function run.<locals>.<lambda> at 0x0000022E71401C60>, {'/przepis/murzynek-z-wisniami': {'ingredient': [{'name': 'mąka pszenna tortowa', 'amount': '160 g - 1 szklanka'}, {'name': 'bardzo duże jajka', 'amount': '2 sztuki - 140 g po rozbiciu'}, {'name': 'czekolada deserowa', 'amount': '100 g - 1 tabliczka'}, {'name': 'masło', 'amount': '120 g'}, {'name': 'erytrol lub mniej cukru', 'amount': '150 g'}, {'name': 'proszek do pieczenia i soda oczyszczona', 'amount': 'po 1 płaskiej łyżeczce'}, {'name': 'wiśnie', 'amount': '300 g'}, {'name': 'mąka pszenna tortowa', 'amount': '160 g - 1 szklanka'}, {'name': 'bardzo duże jajka', 'amount': '2 sztuki - 140 g po rozbiciu'}, {'name': 'czekolada deserowa', 'amount': '100 g - 1 tabliczka'}, {'name': 'masło', 'amount': '120 g'}, {'name': 'erytrol lub mniej cukru', 'amount': '150 g'}, {'name': 'proszek do pieczenia i soda oczyszczona', 'amount': 'po 1 płaskiej łyżeczce'}, {'name': 'wiśnie', 'amount': '300 g'}, {'name': 'mąka pszenna tortowa'